In [13]:

import pandas as pd
import numpy as np
import re

df = pd.read_csv('../data/raw/VN_housing_dataset.csv')
print(f"Kích thước ban đầu: {df.shape}")

# 2. XÓA DỮ LIỆU TRÙNG LẶP (QUAN TRỌNG)
# Web bất động sản có rất nhiều tin spam đăng lại giống hệt nhau
df = df.drop_duplicates()
print(f"Kích thước sau khi xóa trùng lặp: {df.shape}")

cols_to_drop = ['Unnamed: 0', 'Ngày', 'Địa chỉ']
df = df.drop(columns=cols_to_drop)

df.columns = [
    'District', 'Ward', 'House_type', 'Legal', 'Floors', 'Bedrooms', 'Area', 'Length', 'Width', 'Price_per_m2'
]

# Hàm làm sạch số (loại bỏ chữ, đổi dấu phẩy thành chấm)
def extract_number(value):
    if pd.isna(value): return np.nan
    # Chuyển về chuỗi thường
    text = str(value).lower()
    # Nếu có dấu phẩy (86,96) thay bằng chấm (86.96)
    text = text.replace(',', '.')
    # Tìm số (bao gồm cả số thập phân)
    match = re.search(r"[-+]?\d*\.\d+|\d+", text)
    if match:
        return float(match.group())
    return np.nan

# Áp dụng cho các cột số
cols_to_clean = ['Area', 'Length', 'Width', 'Price_per_m2', 'Bedrooms', 'Floors']
for col in cols_to_clean:
    df[col] = df[col].apply(extract_number)

# Tính TỔNG GIÁ TRỊ CĂN NHÀ (Đây là cột quan trọng nhất để dự đoán)
# Công thức: Giá/m2 * Diện tích = Tổng giá (đơn vị: Triệu VNĐ)
df['Total_Price_Billion'] = (df['Price_per_m2'] * df['Area']) / 1000
# (Chia 1000 để đổi từ Triệu sang Tỷ cho số nhỏ dễ nhìn)

# Xử lý cột Quận/Huyện (Bỏ chữ 'Quận', 'Phường')
df['District'] = df['District'].str.replace('Quận', '').str.strip()
df['Ward'] = df['Ward'].str.replace('Phường', '').str.replace('Xã', '').str.strip()

# Xử lý thiếu (Missing Values)
# Xóa dòng nếu thiếu Diện tích, Giá, Dài và Rộng
df = df.dropna(subset=['Area', 'Total_Price_Billion', 'Floors', 'Bedrooms'])

# Cột Pháp lý: Điền 'Dang_cap_nhat'
df['Legal'] = df['Legal'].fillna('Dang_cap_nhat')


# Xóa cột Dài, Rộng (vì thiếu quá nhiều)
df = df.drop(columns=['Length', 'Width'])

# Lọc theo Diện tích
# Chỉ lấy nhà từ 10m2 đến 500m2
df = df[(df['Area'] >= 10) & (df['Area'] <= 500)]

# B. Lọc theo phương pháp IQR cho Đơn giá/m2 (Chuẩn thống kê)
# Giúp loại bỏ những nhà có giá ảo (quá rẻ hoặc quá đắt vô lý)
Q1 = df['Price_per_m2'].quantile(0.25)
Q3 = df['Price_per_m2'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df = df[(df['Price_per_m2'] >= lower_bound) & (df['Price_per_m2'] <= upper_bound)]
print(f"Đã lọc ngoại lai giá/m2. Giới hạn: {lower_bound:.1f} - {upper_bound:.1f} triệu/m2")

# Lọc bỏ số phòng ngủ
# Nếu diện tích < 40m2 mà có >= 8 phòng ngủ -> XÓA (Vì quá chật)
df = df[~((df['Area'] < 40) & (df['Bedrooms'] >= 8))]

# Nếu số tầng < 2 mà có >= 5 phòng ngủ (trừ khi diện tích > 100m2) -> XÓA
df = df[~((df['Floors'] < 2) & (df['Bedrooms'] >= 5) & (df['Area'] < 100))]

# Lọc theo Giá
# Chỉ lấy nhà giá từ 500 Triệu (0.5 tỷ) đến 100 Tỷ
df = df[(df['Total_Price_Billion'] >= 0.5) & (df['Total_Price_Billion'] <= 100)]

# Lưu file sạch
df.to_csv('../data/processed/clean_vn_housing.csv', index=False)

Kích thước ban đầu: (82497, 13)
Kích thước sau khi xóa trùng lặp: (82497, 13)
Đã lọc ngoại lai giá/m2. Giới hạn: 13.4 - 172.0 triệu/m2


In [14]:
df.count()

District               33049
Ward                   33043
House_type             33049
Legal                  33049
Floors                 33049
Bedrooms               33049
Area                   33049
Price_per_m2           33049
Total_Price_Billion    33049
dtype: int64

In [15]:
df.head(10)

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion
0,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4.0,5.0,46.0,86.96,4.00016
2,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,65.00,2.60000
9,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ,4.0,3.0,41.0,64.63,2.64983
11,Hoàng Mai,Định Công,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,30.0,83.33,2.49990
15,Long Biên,Bồ Đề,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,52.0,93.27,4.85004
17,Hoàn Kiếm,Phúc Tân,"Nhà mặt phố, mặt tiền",Đã có sổ,7.0,10.0,165.0,103.03,16.99995
18,Long Biên,Gia Thụy,"Nhà mặt phố, mặt tiền",Đã có sổ,2.0,3.0,70.0,102.86,7.20020
20,Nam Từ Liêm,Phương Canh,"Nhà ngõ, hẻm",Đã có sổ,5.0,3.0,32.0,51.56,1.64992
21,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ,5.0,6.0,65.0,75.38,4.89970
22,Hoàng Mai,Tương Mai,"Nhà ngõ, hẻm",Đã có sổ,5.0,4.0,45.0,64.44,2.89980


In [16]:
# 1. Lưu và mã hóa District
df['District_old'] = df['District'].copy()
df['District'] = pd.factorize(df['District'])[0]

print("--- BẢNG TRA CỨU DISTRICT ---")
district_map = df[['District_old', 'District']].drop_duplicates().sort_values('District')
print(district_map.to_string(index=False)) # Dùng to_string để hiện thị đẹp hơn

--- BẢNG TRA CỨU DISTRICT ---
    District_old  District
        Cầu Giấy         0
    Hai Bà Trưng         1
         Hà Đông         2
       Hoàng Mai         3
       Long Biên         4
       Hoàn Kiếm         5
     Nam Từ Liêm         6
 Huyện Thanh Trì         7
         Ba Đình         8
      Thanh Xuân         9
         Đống Đa        10
     Bắc Từ Liêm        11
          Tây Hồ        12
  Huyện Hoài Đức        13
   Huyện Gia Lâm        14
 Huyện Chương Mỹ        15
  Thị xã Sơn Tây        16
  Huyện Đông Anh        17
Huyện Thường Tín        18
 Huyện Thanh Oai        19
Huyện Đan Phượng        20
  Huyện Quốc Oai        21
   Huyện Sóc Sơn        22
   Huyện Mê Linh        23
Huyện Thạch Thất        24


In [17]:
# 2. Lưu và mã hóa Ward
df['Ward_old'] = df['Ward'].copy()
df['Ward'] = pd.factorize(df['Ward'])[0]

print("--- BẢNG TRA CỨU WARD ---")
ward_map = df[['Ward_old', 'Ward']].drop_duplicates().sort_values('Ward')
print(ward_map.to_string(index=False))

--- BẢNG TRA CỨU WARD ---
          Ward_old  Ward
               NaN    -1
          Nghĩa Đô     0
         Minh Khai     1
          Văn Quán     2
         Định Công     3
             Bồ Đề     4
          Phúc Tân     5
          Gia Thụy     6
       Phương Canh     7
         Tương Mai     8
            La Khê     9
            Tây Mỗ    10
      Tả Thanh Oai    11
          Tam Hiệp    12
           Cống Vị    13
         Bách Khoa    14
         Vĩnh Phúc    15
      Khương Trung    16
          Trung Tự    17
         Kiến Hưng    18
           Láng Hạ    19
         Ô Chợ Dừa    20
          Mai Dịch    21
     Hoàng Văn Thụ    22
          Vạn Phúc    23
       Thanh Lương    24
           Ngọc Hà    25
            Phú Đô    26
         Long Biên    27
          Kim Liên    28
         Phúc Diễn    29
            Kim Mã    30
         Trung Văn    31
       Thượng Đình    32
        Nhân Chính    33
        Trung Liệt    34
        Hoàng Liệt    35
         Mỹ Đình 2    36

In [18]:
# 3. Lưu và mã hóa House_type
df['House_type_old'] = df['House_type'].copy()
df['House_type'] = pd.factorize(df['House_type'])[0]

print("--- BẢNG TRA CỨU HOUSE_TYPE ---")
house_type_map = df[['House_type_old', 'House_type']].drop_duplicates().sort_values('House_type')
print(house_type_map.to_string(index=False))

--- BẢNG TRA CỨU HOUSE_TYPE ---
       House_type_old  House_type
         Nhà ngõ, hẻm           0
Nhà mặt phố, mặt tiền           1
         Nhà biệt thự           2
      Nhà phố liền kề           3


In [19]:
# 4. Lưu và mã hóa Legal
df['Legal_old'] = df['Legal'].copy()
df['Legal'] = pd.factorize(df['Legal'])[0]

print("--- BẢNG TRA CỨU LEGAL ---")
legal_map = df[['Legal_old', 'Legal']].drop_duplicates().sort_values('Legal')
print(legal_map.to_string(index=False))


--- BẢNG TRA CỨU LEGAL ---
    Legal_old  Legal
     Đã có sổ      0
  Đang chờ sổ      1
Dang_cap_nhat      2
 Giấy tờ khác      3


In [20]:
df

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion,District_old,Ward_old,House_type_old,Legal_old
0,0,0,0,0,4.0,5.0,46.0,86.96,4.00016,Cầu Giấy,Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ
2,1,1,0,0,4.0,4.0,40.0,65.00,2.60000,Hai Bà Trưng,Minh Khai,"Nhà ngõ, hẻm",Đã có sổ
9,2,2,0,0,4.0,3.0,41.0,64.63,2.64983,Hà Đông,Văn Quán,"Nhà ngõ, hẻm",Đã có sổ
11,3,3,0,0,5.0,4.0,30.0,83.33,2.49990,Hoàng Mai,Định Công,"Nhà ngõ, hẻm",Đã có sổ
15,4,4,0,0,5.0,4.0,52.0,93.27,4.85004,Long Biên,Bồ Đề,"Nhà ngõ, hẻm",Đã có sổ
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82465,1,1,1,3,4.0,2.0,11.0,61.82,0.68002,Hai Bà Trưng,Minh Khai,"Nhà mặt phố, mặt tiền",Giấy tờ khác
82470,3,3,0,0,5.0,6.0,45.0,96.67,4.35015,Hoàng Mai,Định Công,"Nhà ngõ, hẻm",Đã có sổ
82472,9,66,3,0,4.0,4.0,42.0,71.43,3.00006,Thanh Xuân,Khương Đình,Nhà phố liền kề,Đã có sổ
82479,10,19,1,0,5.0,4.0,35.0,94.29,3.30015,Đống Đa,Láng Hạ,"Nhà mặt phố, mặt tiền",Đã có sổ


In [21]:
# Dùng errors='ignore' để tránh lỗi nếu cột đã bị xóa
cols_to_remove = ['District_old', 'Ward_old', 'House_type_old', 'Legal_old']
df = df.drop(columns=cols_to_remove, errors='ignore')

In [22]:
df

,District,Ward,House_type,Legal,Floors,Bedrooms,Area,Price_per_m2,Total_Price_Billion
0,0,0,0,0,4.0,5.0,46.0,86.96,4.00016
2,1,1,0,0,4.0,4.0,40.0,65.00,2.60000
9,2,2,0,0,4.0,3.0,41.0,64.63,2.64983
11,3,3,0,0,5.0,4.0,30.0,83.33,2.49990
15,4,4,0,0,5.0,4.0,52.0,93.27,4.85004
...,...,...,...,...,...,...,...,...,...
82465,1,1,1,3,4.0,2.0,11.0,61.82,0.68002
82470,3,3,0,0,5.0,6.0,45.0,96.67,4.35015
82472,9,66,3,0,4.0,4.0,42.0,71.43,3.00006
82479,10,19,1,0,5.0,4.0,35.0,94.29,3.30015


In [23]:
df.to_csv('../data/processed/clean_vn_housing.csv', index=False)